In [5]:
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
from sqlalchemy.orm import declarative_base
# from sqlalchemy.ext.declarative import declarative_base

DATABASE_URL = "sqlite:///database.db"

engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Define the User table using SQLAlchemy ORM
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    first_name = Column(String, index=True)
    last_name = Column(String, index=True)
    email = Column(String, unique=True, index=True)

# Create the table if it does not exist
Base.metadata.create_all(bind=engine)

# Assuming create_user function and UserCreate schema are defined correctly
# from app.db.crud import create_user
# from app.db.schemas import UserCreate

from pydantic import BaseModel

class UserBase(BaseModel):
    email: str
    first_name: str = None
    last_name: str = None

class UserCreate(UserBase):
    password: str

    class Config:
        orm_mode = True

def create_user(db: Session, user: UserBase):
    db_user = User(
        first_name=user.first_name,
        last_name=user.last_name,
        email=user.email,
    )
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    return db_user

def init() -> None:
    db = SessionLocal()
    first_name = "test-first_name"
    last_name = "test-last_name"
    email = "test-email"

    create_user(
        db,
        UserBase(
            first_name=first_name,
            last_name=last_name,
            email=email
        ),
    )
    db.commit()

if __name__ == "__main__":
    print("Creating user")
    init()
    print("Superuser created")

Creating user


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: users.email
[SQL: INSERT INTO users (first_name, last_name, email) VALUES (?, ?, ?)]
[parameters: ('test-first_name', 'test-last_name', 'test-email')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [15]:
import typing as t

class UserOut(UserBase):
    pass

def get_users(
    db: Session, skip: int = 0, limit: int = 100
):
    return db.query(User).offset(skip).limit(limit).all()



db = SessionLocal()
x = get_users(db)
x

In [18]:
x[0].__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x10fa0af90>,
 'id': 1,
 'last_name': 'test-last_name',
 'first_name': 'test-first_name',
 'email': 'test-email'}

In [19]:
type(x[0])


__main__.User

In [13]:
for i in x:
    print(i)
